<h2><center>Final Project: Where it Pays to Attend College</center></h2>

<h3><center>By Ansh Vanam and Rishi Mullangi<center></h3>

<h2>Introduction</h2>

For our final project, we have decided to work with a dataset that contains salaries of various college graduates based on major, type of school (private, public), and region. The dataset we chose has three separate CSV files, "salaries-by-college-type.csv", "salaries-by-region.csv", and "degrees-that-pay-back.csv". These CSV files contain data from graduating up to their mid-career, including percentiles ranging from 10-90. 

Using these CSV files, we plan to look into how these variables affect the salaries of students, and how these factors can affect choices of students looking to go into college. After that, we plan to make a prediction model that will project average Mid-Career  salaries based on region and starting median salaries. 

Our first step is to navigate to this Kaggle dataset, created by the Wall Street Journal:

https://www.kaggle.com/datasets/wsj/college-salaries/code?resource=download&select=salaries-by-college-type.csv

From here, we can download the three CSV files and import them into our code editor. Then, we use pandas to create dataframes out of the CSV files. 

Note: Ensure you have the following python libraries installed on your system and the CSV files downloaded to run the tutorial.

Pandas documentation - https://pandas.pydata.org/docs/ 

Seaborn documentation - https://seaborn.pydata.org

Numpy documentation - https://numpy.org/doc/ 

Matplotlib documentation - https://matplotlib.org/stable/index.html 

In [ ]:
#Importing our required libraries
import pandas as pd
import numpy as np
import statsmodels.api as stm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm

<h2>Obtaining and Cleaning Data</h2>

The first table we're looking at is salaries-by-college-type, which details the names of various colleges as well as their School Type. In the next python block we will work on cleaning the data and making it more useable. First, we need to change the values to floats so that we can do operations on them. Then, we change the names of the columns to be more clear and correspond well with the other dataframes we will initialize.

In [ ]:
#Reading in the CSV file and converting it to a dataframe using pandas
df_by_colleges = pd.read_csv('salaries-by-college-type.csv')

#Getting rid of $ signs as well as commas so that we can convert values to floats. 
for (columnName, columnData) in df_by_colleges.iteritems():
    df_by_colleges[columnName] = df_by_colleges[columnName].str.replace('$','')
    df_by_colleges[columnName] = df_by_colleges[columnName].str.replace(',', '')
df_by_colleges = df_by_colleges.astype({'Starting Median Salary':'float', 
'Mid-Career Median Salary': 'float', 'Mid-Career 10th Percentile Salary': 'float', 
'Mid-Career 25th Percentile Salary':'float', 'Mid-Career 75th Percentile Salary':'float', 
'Mid-Career 90th Percentile Salary':'float'})

print(df_by_colleges.dtypes)

#Rename columns to be more concise but still descriptive 
df_by_colleges = df_by_colleges.rename(columns = {'School Name':'School Name',
'School Type':'School Type','Starting Median Salary':'Starting Median', 
'Mid-Career Median Salary':'Mid-Career Median', 
'Mid-Career 10th Percentile Salary':'Mid-Career 10%',
'Mid-Career 25th Percentile Salary':'Mid-Career 25%', 
'Mid-Career 75th Percentile Salary':'Mid-Career 75%', 
'Mid-Career 90th Percentile Salary':'Mid-Career 90%'})
df_by_colleges.head()

Now that we've cleaned up our first table by converting the values that are numerical to floats, we can move on to the next table, 'degrees-that-pay-back.csv', which details undergraduate major's median starting salary in relation to their undergraduate major, mid-career median salary, and other useful information that we can use later on in the tutorial. We will stick to the same column renaming conventions so that the column names remain aligned across the different dataframes

In [ ]:
#Reading in the CSV file and converting it to a dataframe using pandas
df_by_major = pd.read_csv('degrees-that-pay-back.csv')

#Changing all of the columns to floats (Where required)
for (columnName, columnData) in df_by_major.iteritems():
    if(columnName == 'Percent change from Starting to Mid-Career Salary'): continue
    df_by_major[columnName] = df_by_major[columnName].str.replace('$','')
    df_by_major[columnName] = df_by_major[columnName].str.replace(',', '')
df_by_major = df_by_major.astype({'Starting Median Salary':'float', 
'Mid-Career Median Salary': 'float', 'Mid-Career 10th Percentile Salary': 'float', 
'Mid-Career 25th Percentile Salary':'float', 'Mid-Career 75th Percentile Salary':'float', 
'Mid-Career 90th Percentile Salary':'float'})

print(df_by_major.dtypes)

#Rename columns to be more concise but still descriptive 
df_by_major = df_by_major.rename(columns = {'Undergraduate Major':'Major',
'Starting Median Salary':'Starting Median', 
'Mid-Career Median Salary':'Mid-Career Median', 
'Percent change from Starting to Mid-Career Salary': 'Start to Mid % Change',
'Mid-Career 10th Percentile Salary':'Mid-Career 10%',
'Mid-Career 25th Percentile Salary':'Mid-Career 25%', 
'Mid-Career 75th Percentile Salary':'Mid-Career 75%', 
'Mid-Career 90th Percentile Salary':'Mid-Career 90%'})
df_by_major.head()

Finally, we can look at the last CSV file, 'salaries-by-region.csv', which details college graduate salaries by region and college. We will perform the same operations on this dataframe, namely, converting values to floats as appropriate, and renaming columns for compatibility across the dataframes.

In [ ]:
#Reading in the CSV file and converting it to a dataframe using pandas
df_by_region = pd.read_csv('salaries-by-region.csv')

#Converting numerical columns to float values.
for (columnName, columnData) in df_by_region.iteritems():
    if(columnName == 'Percent change from Starting to Mid-Career Salary'): continue
    df_by_region[columnName] = df_by_region[columnName].str.replace('$','')
    df_by_region[columnName] = df_by_region[columnName].str.replace(',', '')
df_by_region = df_by_region.astype({'Starting Median Salary':'float', 
'Mid-Career Median Salary': 'float', 'Mid-Career 10th Percentile Salary': 'float', 
'Mid-Career 25th Percentile Salary':'float', 'Mid-Career 75th Percentile Salary':'float', 
'Mid-Career 90th Percentile Salary':'float'})

print(df_by_region.dtypes)

#Rename columns to be more concise but still descriptive 
df_by_region = df_by_region.rename(columns = {'School Name':'School Name',
'Starting Median Salary':'Starting Median', 
'Mid-Career Median Salary':'Mid-Career Median', 
'Mid-Career 10th Percentile Salary':'Mid-Career 10%',
'Mid-Career 25th Percentile Salary':'Mid-Career 25%', 
'Mid-Career 75th Percentile Salary':'Mid-Career 75%', 
'Mid-Career 90th Percentile Salary':'Mid-Career 90%'})
df_by_region.head()

<h2>Data Management/Representation</h2>
Next, we will represent various data from the dataframes to try and understand the tables more, and focus our tutorial. These graphs will provide a visual understanding of what all the dataframes represent.

First, we want to understand the relationship between region and starting median salary, to see if the region has a heavy effect on new graduates' salaries. To do this, we can create a simple bar graph that will display the median salary ranges of the various regions. 

In [ ]:
#plt.figure(figsize = (10,7))
plt.title('Region and Starting Median Salary')
plt.xlabel('Region')
plt.ylabel('Starting Median Salary')
plt.bar(df_by_region['Region'], df_by_region['Starting Median'])
plt.plot()

Here, we see that California universities have the highest starting median salary, followed closely by colleges in the Northeast. 

It is likely that a high percentage of students decide to live within the same region/state where they went to school. Since California and the Northeast are more expensive to live in than the other three regions, it makes sense that the starting median salary for students attending universities in either of the two regions is also higher. 

California and the Northeast might also have a higher percentage of prestigious schools than the other regions, which lead to better job offers as well as more companies hiring from these locations. Since California is considered a tech hub, it makes sense that their starting salary is the highest of the five regions, as lots of major tech companies that pay well hire from Californian schools. 

Now we will replicate the Median Salary vs Region graph, except for Mid Career instead of Starting, to see how careers progress over time in the various regions. 

In [ ]:
#plt.figure(figsize = (10,7))
plt.title('Region and Mid-Career Median Salary')
plt.xlabel('Region')
plt.ylabel('Mid-Career Median Salary')
plt.bar(df_by_region['Region'], df_by_region['Mid-Career Median'])
plt.plot()

Here we see that although California universities have the highest starting median salary, Northeastern universities have the highest mid career median salary. This gives us insight into how although Californian schools boast a high starting wage, a worker may be better off shifting to the Northeast region if they want to maximize their pay during their Mid-Career years. 

School Type vs. Starting Median

In [ ]:
#plt.figure(figsize = (10,7))
plt.title('School Type and Starting Median Salary')
plt.xlabel('School Type')
plt.ylabel('Starting Median Salary')
plt.bar(df_by_colleges['School Type'], df_by_colleges['Starting Median'])
plt.plot()

Here, we can see that the Engineering starting salary is the highest, beating out Ivy League schools. 

School Type vs. Mid-Career Median

In [ ]:
#plt.figure(figsize = (10,7))
plt.title('School Type and Mid-Career Median Salary')
plt.xlabel('School Type')
plt.ylabel('Mid-Career Median Salary')
plt.bar(df_by_colleges['School Type'], df_by_colleges['Mid-Career Median'])
plt.plot()

We notice that the starting median salary from Engineering schools is higher than that of Ivy League, but the mid career median salary of Engineering schools is lower than that of Ivy League. This is a trend similar to what we noticed in California-Northeastern school salaries.

Major vs Salary analysis

In [ ]:
#Initial analysis on major - salary.
    #Theres a lot of majors so I couldnt think of a way to graph this
    #For now I just made two lists sorted in descending order of start and mid salary respectively

#Start
starting_order = df_by_major.sort_values(by = 'Starting Median', ascending = False)['Major'].reset_index(drop = True)
starting_order

In [ ]:
#Create Scatterplot

For our final data analysis representation, we wanted to showcase the starting, mid-career, and 90th percentile mid-career salaries as time goes on using a boxplot, as this would show outliers, median, and the varying quartiles

In [ ]:
plt.figure(figsize=(8,8))
lst = [df_by_major['Starting Median'], df_by_major['Mid-Career Median'], df_by_major['Mid-Career 90%']]

plt.boxplot(lst)
plt.ylabel('Salary')
plt.xlabel('Starting, Mid-Career, and Mid-Career 90th percentile')
plt.show()

Here we see that starting salaries have the shortest standard deviation, followed by mid-career, and mid-career 90th percentile has the largest standard deviation with a large range. There is also an outlier in the starting salaries: ~75,000, which is greater even than the median of the mid-career salaries

In [ ]:
#Finding out which major has the highest starting median salary
df_by_major[df_by_major['Starting Median'] == df_by_major['Starting Median'].max()]

We notice that the start to mid-career salary change for physician assistant is 23.4%

In [ ]:
#Finding the mean mid-career median salary
df_by_major['Mid-Career Median'].mean()

In [ ]:
#Finding the mean percentage change from starting to midcareer salaries for all majors
df_by_major['Start to Mid % Change'].mean()

Although the physician assistant starting median salary is way higher than the starting salaries for the other majors, we see that the 'start to mid career percentage change' in salary of physician assistant is 23.4%, which is a lot lower than that of the mean 'start to mid career percentage change' of 69.3%

<h2>Predicting Mid-Career Median Salary based on School Type and Starting Salary</h2>

For the next part of our tutorial, we will be attempting to make a model that predicts Mid-Career Median salaries based on the school type that the student is attending and their projected starting salary. To understand how the model might end up looking, we want to create a regression line that can show us potential predictions. 

In [ ]:
# g = sns.FacetGrid(data, row = 'continent', aspect = 2)
# g = g.map(plt.scatter, x= data['year'], y = data['lifeExp'])

# plt.show()

#Using groupby and seaborn to create a single graph with all the regression lines contained
df_by_colleges = df_by_colleges.rename(columns = {"Starting Median":"Starting_Median", 
"Mid-Career Median":"Mid_Career_Median"})
figure(figsize=(10,8))
for name, group in df_by_colleges.groupby('School Type'):
    plt.scatter(group.Starting_Median, group.Mid_Career_Median, label = name)
    sns.regplot(x = group.Starting_Median, y = group.Mid_Career_Median, data = df_by_colleges, ci=None)

plt.title('Expected Mid Career Median Salary')
plt.xlabel('Starting Median Salary')
plt.ylabel('Mid-Career Salary')
plt.legend()
plt.show()

# sns.scatterplot(data['year'], data['lifeExp'], hue = data['continent'])
# plt.show()

The Ivy League schools regression line has a higher intersect (higher starting salaries than other schools) and lower slope (less increase in mid career salary from an increase in starting salary). This could partly be due to a lesser number of Ivy League schools, meaning a smaller number of datapoints. Liberal Arts schools have the highest regression line slope, meaning that a higher starting salary goes a longer way in getting a higher mid career salary

In [ ]:
figure(figsize=(10,8))
plt.scatter(group.Starting_Median, group.Mid_Career_Median, label = name)
sns.regplot(x = group.Starting_Median, y = group.Mid_Career_Median, data = df_by_colleges, ci=None)

Regression line when all the data is not split up by school type

In [151]:
X_train, X_test, y_train, y_test = train_test_split(df_by_colleges[['School Type', 'Starting_Median']], 
df_by_colleges['Mid_Career_Median'],random_state=42)

clf = svm.LinearSVR
